In [51]:
import gately
import shapley
import numpy as np
import random
import timeit
import shap

In [52]:
#X, y = shap.datasets.nhanesi()

In [53]:
X, y = shap.datasets.iris()
X = X.values

In [54]:
X.shape

(150, 4)

In [55]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import xgboost

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5)

regr = xgboost.XGBRegressor().fit(X_train, y_train)
regr.fit(X_train, y_train)
regr.score(X_test,y_test)

0.8561962857258385

In [56]:
X_train.shape

(75, 4)

In [57]:
matches = 0

for ind in range(0,20):
    #explainer = shap.KernelExplainer(regr.predict, X_test[ind+1].reshape(1,-1))
    #shap_values = explainer.shap_values(X_test[ind].reshape(1,-1), nsamples=100)
    att_gately, dist_gately = gately.Gately_distribution(X_test[ind],X_test[ind+1],regr.predict)

    """att_shap = {}
    for i in range(0,X.shape[1]):
        att_shap[i] = np.abs(shap_values[0][i])
    
    ks_sorted = sorted(att_shap, key=att_shap.get, reverse=True)
    gate_sorted = sorted(att_gately, key=att_gately.get, reverse=True)
    tick = 0
    
 

    for i,j in zip(ks_sorted,gate_sorted):
 
        if i != j:
            tick = 1
    if tick == 0:
        matches += 1"""
    
    #distributions_shap.append(dist_shap)
    #distributions_gately.append(dist_gately)
        
#distributions_shap = np.asarray(distributions_shap)
#distributions_gately = np.asarray(distributions_gately)
#print(distributions_shap.shape)
#distributions_shap = distributions_shap.reshape(-1,distributions_shap.shape[2])
# = distributions_gately.reshape(-1,distributions_gately.shape[2])

In [58]:
distributions_shap = []
for _ in range(0,1000):
    i = np.random.randint(0,X.shape[0]-1)
    sample = X[i].copy()
    ref = X[i+1]
    
    number_inds = int(np.random.uniform(1,X.shape[1]-1))
    
    selected_indexes = []
    for ind in range(0,number_inds):
        selected_indexes.append(int(np.random.uniform(0,X.shape[1])))
    
    sample[selected_indexes] = ref[selected_indexes]
    distributions_shap.append(sample)
distributions_shap = np.asarray(distributions_shap)

In [59]:

distributions_gately = []
for _ in range(0,1000):
    
    number_inds = int(np.random.randint(0,2))
    i = np.random.randint(0,X.shape[0]-1)
    if number_inds == 0:
        selected_ind =  int(np.random.uniform(0,X.shape[1]))
        sample = X[i].copy()
        ref = X[i+1]
        sample[selected_ind] = ref[selected_ind]
        distributions_gately.append(sample)
    elif number_inds == 1:
        selected_ind =  int(np.random.uniform(0,X.shape[1]))
        sample = X[i+1].copy()
        ref = X[i]
        sample[selected_ind] = ref[selected_ind]
        distributions_gately.append(sample)
                      
distributions_gately = np.asarray(distributions_gately)


In [60]:
samples = X.copy()
samples.shape

(150, 4)

In [88]:
scores_gately = []
scores_shap = []

for i in range(0,10):
    X = np.concatenate([distributions_gately[0:100],samples[0:100]])
    Y = np.concatenate([np.zeros(100),np.ones(100)])
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.model_selection import train_test_split

    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)

    regr = RandomForestClassifier(n_estimators=100, max_depth=10)
    regr.fit(X_train, y_train)
    scores_gately.append(regr.score(X_test,y_test))

    X = np.concatenate([distributions_shap[0:100],samples[0:100]])
    Y = np.concatenate([np.zeros(100),np.ones(100)])

    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)

    regr = RandomForestClassifier(n_estimators=100, max_depth=10)
    regr.fit(X_train, y_train)
    scores_shap.append(regr.score(X_test,y_test))


In [89]:
np.mean(scores_gately)

0.5349999999999999

In [90]:
np.mean(scores_shap)

0.5633333333333332